## Introduction
##### To ensure optimal support for informed business decisions, it is imperative to design a robust database schema that facilitates seamless querying of information. In this exercise, you will assist a company that sells IT accessories in organizing their data effectively, thereby contributing to enhancing their decision-making processes.

In [1]:
#Import Libraries
import sqlite3
import pandas as pd

### Help function

In [2]:
db= "sales.db"

#Run queries on database
def run_query(my_query):
    with sqlite3.connect(db) as conn:
        return pd.read_sql(my_query, conn)

#Execute queries on database
def run_command(my_command):
    with sqlite3.connect(db) as conn:
        conn.isolation_level= None
        conn.execute(my_command)
    print("✔️ SUCCESS")
    
#Insert a dataframe into a table
def insert_dataframe(df, table_name):
    with sqlite3.connect(db) as conn:
        table_columns= run_query(f"SELECT * FROM {table_name} LIMIT 0").columns
        for col in table_columns:
            if col not in df.columns:
                df[col]= None
        df_filtered= df[table_columns]
        df_filtered.to_sql(table_name, conn, if_exists="append", index=False)
        print(f"✔️Data Successfully inserted into {table_name}")

### Load CSV Files into DataFrames

In [3]:
customers= pd.read_csv("C:/Users/Jean Paul/Desktop/Mine/Data Science/Exercises/Datasets/customers.csv")
customers.head(1)

,first_name,last_name,country,phone_number,sales_contact_code
0,Anita,Ruiz,Canada,250766666657,3


In [4]:
employees= pd.read_csv("C:/Users/Jean Paul/Desktop/Mine/Data Science/Exercises/Datasets/employees.csv")
employees.head(1)

,employee_number,last_name,first_name,line_manager,title
0,1,Ross,Shannon,NaN,General Manager


In [5]:
products= pd.read_csv("C:/Users/Jean Paul/Desktop/Mine/Data Science/Exercises/Datasets/products.csv")
products.tail(1)

,product_number,product_name,unit_price
9,10,Wireless Keyboard,99


In [6]:
invoices= pd.read_csv("C:/Users/Jean Paul/Desktop/Mine/Data Science/Exercises/Datasets/invoices.csv")
invoices.head(1)

,invoice_number,invoice_date,billing_phone,total
0,16,2021-09-10,250766666601,1980.0


In [7]:
invoice_items= pd.read_csv("C:/Users/Jean Paul/Desktop/Mine/Data Science/Exercises/Datasets/invoice_items.csv")
invoice_items.head(1)

,invoice_id,product_code,quantity
0,1,7,1


## Question 1: Database Schema
##### Could you kindly explore the provided flat files and propose a well-structred database schema? Emphasize ease of data retrieval and analysis, ensuring data integrity and storage efficiency.

In [12]:
# Question 1: Database Schema
# Defining SQL schema for creating tables

customers_schema = """
CREATE TABLE IF NOT EXISTS Customers (
    phone_number TEXT PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    country TEXT,
    sales_contact_code INTEGER
);
"""

employees_schema = """
CREATE TABLE IF NOT EXISTS Employees (
    employee_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    title TEXT,
    manager_id INTEGER,
    FOREIGN KEY(manager_id) REFERENCES Employees(employee_id)
);
"""

products_schema = """
CREATE TABLE IF NOT EXISTS Products (
    product_number INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    unit_price INTEGER NOT NULL
);
"""

invoices_schema = """
CREATE TABLE IF NOT EXISTS Invoices (
    invoice_number INTEGER PRIMARY KEY,
    invoice_date DATE NOT NULL,
    billing_phone TEXT NOT NULL,
    total REAL NOT NULL,
    FOREIGN KEY(billing_phone) REFERENCES Customers(phone_number)
);
"""

invoice_items_schema = """
CREATE TABLE IF NOT EXISTS Invoice_Items (
    invoice_id INTEGER NOT NULL,
    product_code INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    PRIMARY KEY (invoice_id, product_code),
    FOREIGN KEY(invoice_id) REFERENCES Invoices(invoice_number),
    FOREIGN KEY(product_code) REFERENCES Products(product_number)
);
"""

## Question 2: Create the tables into the database
##### We are going to use the sqlite database engine that is built-in within python environment. You are required to use the appropriate provided helper function provided to create the tables as per ypur design schema. Write and execute necessary commands below.

In [13]:

# Define the database name
db = "sales.db"

# NB: I used Helper functions for interacting with the SQLite database

# NB: Create tables using the schema from Question 1

# Create Customers table
run_command("""
CREATE TABLE IF NOT EXISTS Customers (
    phone_number TEXT PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    country TEXT,
    sales_contact_code INTEGER
);
""")

# Create Employees table
run_command("""
CREATE TABLE IF NOT EXISTS Employees (
    employee_id INTEGER PRIMARY KEY,
    first_name TEXT NOT NULL,
    last_name TEXT NOT NULL,
    title TEXT,
    manager_id INTEGER,
    FOREIGN KEY(manager_id) REFERENCES Employees(employee_id)
);
""")

# Create Products table
run_command("""
CREATE TABLE IF NOT EXISTS Products (
    product_number INTEGER PRIMARY KEY,
    product_name TEXT NOT NULL,
    unit_price INTEGER NOT NULL
);
""")

# Create Invoices table
run_command("""
CREATE TABLE IF NOT EXISTS Invoices (
    invoice_number INTEGER PRIMARY KEY,
    invoice_date DATE NOT NULL,
    billing_phone TEXT NOT NULL,
    total REAL NOT NULL,
    FOREIGN KEY(billing_phone) REFERENCES Customers(phone_number)
);
""")

# Create Invoice_Items table
run_command("""
CREATE TABLE IF NOT EXISTS Invoice_Items (
    invoice_id INTEGER NOT NULL,
    product_code INTEGER NOT NULL,
    quantity INTEGER NOT NULL,
    PRIMARY KEY (invoice_id, product_code),
    FOREIGN KEY(invoice_id) REFERENCES Invoices(invoice_number),
    FOREIGN KEY(product_code) REFERENCES Products(product_number)
);
""")

print("✔️ All tables created successfully!")


✔️ SUCCESS
✔️ SUCCESS
✔️ SUCCESS
✔️ SUCCESS
✔️ SUCCESS
✔️ All tables created successfully!


### Checking the Column Mismatch: DataFrame Columns vs Table Columns in the Database
##### example for: Customers

In [14]:
print(customers.columns)
print(run_query("SELECT * FROM Customers LIMIT 0").columns)

Index(['first_name', 'last_name', 'country', 'phone_number',
       'sales_contact_code'],
      dtype='object')
Index(['phone_number', 'first_name', 'last_name', 'country',
       'sales_contact_code'],
      dtype='object')


### Adjust DataFrame Columns to Match Table Columns

In [15]:
# Ensure DataFrame columns match table columns
customers = customers[['first_name', 'last_name', 'country', 'phone_number', 'sales_contact_code']]
employees = employees[['employee_number', 'last_name', 'first_name', 'line_manager', 'title']]
products = products[['product_number', 'product_name', 'unit_price']]
invoices = invoices[['invoice_number', 'invoice_date', 'billing_phone', 'total']]
invoice_items = invoice_items[['invoice_id', 'product_code', 'quantity']]

### Insert DataFrames into Database Tables

In [16]:
insert_dataframe(customers, "Customers")
insert_dataframe(employees, "Employees")
insert_dataframe(products, "Products")
insert_dataframe(invoices, "Invoices")
insert_dataframe(invoice_items, "Invoice_Items")

✔️Data Successfully inserted into Customers
✔️Data Successfully inserted into Employees
✔️Data Successfully inserted into Products
✔️Data Successfully inserted into Invoices
✔️Data Successfully inserted into Invoice_Items


### Make query to verify if data are database

In [17]:
run_query("SELECT * FROM Customers LIMIT 2")

,phone_number,first_name,last_name,country,sales_contact_code
0,250766666657,Anita,Ruiz,Canada,3
1,250766666656,Kimberly,Taylor,United Kingdom,3


In [18]:
run_query("SELECT * FROM Products LIMIT 2")

,product_number,product_name,unit_price
0,1,External Hard Drive,99
1,2,Portable Bluetooth Speaker,99


In [ ]:
## Question3: Employees and their Supervisors
##### Now that the tables are created and files are loaded into the database, kindly provide a list of all employees and their respective line managers in the format below.

|EMPLOYEE FULL NAME|EMPLOYEE TITLE|MANAGER FULL NAME|MANAGER TITLE|
|---|---|---|---|
|---|---|---|---|

In [26]:
query = '''
SELECT 
    emp.first_name || ' ' || emp.last_name AS employee_full_name,
    emp.title AS employee_title,
    mgr.first_name || ' ' || mgr.last_name AS manager_full_name,
    mgr.title AS manager_title
FROM 
    Employees AS emp
LEFT JOIN 
    Employees AS mgr ON emp.manager_id = mgr.employee_id;
'''

# Run the query and fetch the results
employee_manager_df = run_query(query)
employee_manager_df.head(5)


,employee_full_name,employee_title,manager_full_name,manager_title
0,Shannon Ross,General Manager,None,None
1,Mark Griffin,Sales Manager,None,None
2,Patricia Estrada,Sales Officer,None,None
3,Alicia West,Sales Officer,None,None
4,Cassidy Petersen,Sales Officer,None,None


## Question 4: Employees Performance
##### Each customer is assigned to a sales officer, provide a query that would help to analyze if any sales officer is performing either better or worse than the others. Also shows the resulted table.

In [31]:
query = '''
SELECT 
    e.first_name || ' ' || e.last_name AS sales_officer_full_name,e.title AS sales_officer_title,
    e.title AS sales_officer_title,
    COUNT(DISTINCT c.phone_number) AS number_of_customers,
    SUM(i.total) AS total_sales
FROM 
    Customers AS c
JOIN 
    Employees AS e ON c.sales_contact_code = e.employee_id
JOIN 
    Invoices AS i ON c.phone_number = i.billing_phone
WHERE 
    c.country LIKE 'Canada%'  -- Filter by country, e.g., "Canada"
GROUP BY 
    e.employee_id
HAVING 
    SUM(i.total) > 5000  -- Only include sales officers with total sales > 5000
ORDER BY 
    total_sales DESC;
'''

# Run the query and fetch the results
sales_performance_df = run_query(query)
sales_performance_df.head()


,sales_officer_full_name,sales_officer_title,sales_officer_title,number_of_customers,total_sales
0,Patricia Estrada,Sales Officer,Sales Officer,3,25839.0
1,Alicia West,Sales Officer,Sales Officer,3,24453.0
2,Cassidy Petersen,Sales Officer,Sales Officer,1,7029.0


## Question 5: Products Performance
##### The Marketing Team has hired a Data Analyst Intern who will need to regularly report on the sales performance of various products to facilitate increased advertising or recommend discounts. Create a view that shows the total quantity and amount for each product that the intern will query. Display the results.

In [38]:
# SQL query to create the Product_Performance view
create_view_query = '''
CREATE VIEW IF NOT EXISTS Product_Performance AS
SELECT 
    p.product_name,
    SUM(i.quantity) AS total_quantity_sold,
    SUM(i.quantity * p.unit_price) AS total_amount_generated
FROM 
    Invoice_Items AS i
LEFT JOIN 
    Products AS p ON i.product_code = p.product_number
GROUP BY 
    p.product_name
ORDER BY 
    total_sales_amount DESC;
'''

# Execute the query to create the view
run_command(create_view_query)

# Query to select data from the view
query = '''
SELECT * FROM Product_Performance;
'''

# Run the query and fetch the results
product_performance_df = run_query(query)
product_performance_df.head()


✔️ SUCCESS


,product_name,total_quantity_sold,total_amount_generated
0,External Hard Drive,252,24948
1,Flash Drive,265,26235
2,Laptop Stand,271,26829
3,Portable Bluetooth Speaker,267,26433
4,Power Bank,262,25938
